In [1]:
try:
  import pandas as pd
  from nltk import word_tokenize
  import nltk
  from spellchecker import SpellChecker
  from nltk.corpus import stopwords
  from nltk.tokenize import RegexpTokenizer
  from nltk import FreqDist
  from nltk.stem import WordNetLemmatizer
  from nltk import pos_tag
  from nltk.stem import PorterStemmer
  import re
except:
  import pandas as pd
  from nltk import word_tokenize
  import nltk
  from spellchecker import SpellChecker
  from nltk.corpus import stopwords
  from nltk.tokenize import RegexpTokenizer
  from nltk import FreqDist
  from nltk.stem import WordNetLemmatizer
  from nltk import pos_tag
  from nltk.stem import PorterStemmer
  import re
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\陳柏硯\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\陳柏硯\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [2]:
# 讀取TSV檔案到pandas DataFrame
df = pd.read_csv('all_annotated.tsv', sep="\t")

# 確認欄位名稱
print(df.columns)
df_text = df[['Tweet']]
# df_text=df_text.iloc[:1000]
# 將Tweet轉為小寫
df_text['Tweet']=df_text['Tweet'].str.lower()

# 移除空白
def remove_whitespace(text):
    return  " ".join(text.split())

# 測試
text = " We will going to win this match"
remove_whitespace(text)

# 分詞
df_text['Tweet']=df_text['Tweet'].apply(lambda X: word_tokenize(X))
df_text.head()

C:\Users\陳柏硯\AppData\Local\Temp\ipykernel_42156\1203838967.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['Tweet']=df_text['Tweet'].str.lower()


Index(['Tweet ID', 'Country', 'Date', 'Tweet', 'Definitely English',
       'Ambiguous', 'Definitely Not English', 'Code-Switched',
       'Ambiguous due to Named Entities', 'Automatically Generated Tweets'],
      dtype='object')


C:\Users\陳柏硯\AppData\Local\Temp\ipykernel_42156\1203838967.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['Tweet']=df_text['Tweet'].apply(lambda X: word_tokenize(X))


,Tweet
0,"[bugün, bulusmami, lazimdiii]"
1,"[volkan, konak, adami, tribe, sokar, yemin, ed..."
2,[bed]
3,"[i, felt, my, first, flash, of, violence, at, ..."
4,"[ladies, drink, and, get, in, free, till, 10:30]"


In [3]:
# 拼字檢
def spell_check(text):
    result = []
    spell = SpellChecker()
    for word in text:
        correct_word = spell.correction(word)
        result.append(correct_word)
    return result

# 測試
text = "confuson matrx".split()
spell_check(text)

['confusion', 'matrix']

In [4]:
df_text['Tweet'] = df_text['Tweet'].apply(spell_check)
df_text.head()
# 461min.....

C:\Users\陳柏硯\AppData\Local\Temp\ipykernel_42156\3039587328.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['Tweet'] = df_text['Tweet'].apply(spell_check)


,Tweet
0,"[burn, None, None]"
1,"[vodka, knock, damn, tribe, solar, hemin, eeri..."
2,[bed]
3,"[i, felt, my, first, flash, of, violence, at, ..."
4,"[ladies, drink, and, get, in, free, till, None]"


In [5]:
# 停用字移除
print(stopwords.words('english'))

en_stopwords = stopwords.words('english')

def remove_stopwords(text):
    result = []
    for token in text:
        if token not in en_stopwords:
            result.append(token)
    return result

# 測試
text = "this is the only solution of that question".split()
remove_stopwords(text)

df_text['Tweet'] = df_text['Tweet'].apply(remove_stopwords)
df_text.head()

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

C:\Users\陳柏硯\AppData\Local\Temp\ipykernel_42156\1179568509.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['Tweet'] = df_text['Tweet'].apply(remove_stopwords)


,Tweet
0,"[burn, None, None]"
1,"[vodka, knock, damn, tribe, solar, hemin, eeri..."
2,[bed]
3,"[felt, first, flash, violence, fool, bumped, N..."
4,"[ladies, drink, get, free, till, None]"


In [6]:
# 移除標點符號
def remove_punct(text):
    if text is None:
        return ""
    text = str(text)  # 將 text 轉換為字串類型
    tokenizer = RegexpTokenizer(r"\w+")
    lst = tokenizer.tokenize(text)
    return lst


# 測試
text=df_text['Tweet'][0]
print(text)
remove_punct(text)

df_text['Tweet'] = df_text['Tweet'].apply(remove_punct)
df_text.head()

['burn', None, None]


C:\Users\陳柏硯\AppData\Local\Temp\ipykernel_42156\1179465151.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['Tweet'] = df_text['Tweet'].apply(remove_punct)


,Tweet
0,"[burn, None, None]"
1,"[vodka, knock, damn, tribe, solar, hemin, eerie]"
2,[bed]
3,"[felt, first, flash, violence, fool, bumped, N..."
4,"[ladies, drink, get, free, till, None]"


In [7]:
# 常見字詞
def frequent_words(df):
    lst=[]
    for text in df.values:
        lst+=text[0]
    fdist=FreqDist(lst)
    return fdist.most_common(10)

freq_words = frequent_words(df_text)

lst = []
for a,b in freq_words:
    lst.append(b)

def remove_freq_words(text):
    result=[]
    for item in text:
        if item not in lst:
            result.append(item)
    return result

df_text['Tweet']=df_text['Tweet'].apply(remove_freq_words)
df_text.head()

C:\Users\陳柏硯\AppData\Local\Temp\ipykernel_42156\2037203970.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['Tweet']=df_text['Tweet'].apply(remove_freq_words)


,Tweet
0,"[burn, None, None]"
1,"[vodka, knock, damn, tribe, solar, hemin, eerie]"
2,[bed]
3,"[felt, first, flash, violence, fool, bumped, N..."
4,"[ladies, drink, get, free, till, None]"


In [8]:
# 詞性還原
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

def lemmatization(text):
    result=[]
    wordnet = WordNetLemmatizer()
    for token,tag in pos_tag(text):
        pos=tag[0].lower()
        if pos not in ['a', 'r', 'n', 'v']:
            pos='n'
        result.append(wordnet.lemmatize(token,pos))
    return result

# 測試
text = ['running','ran','runs']
lemmatization(text)

df_text['Tweet']=df_text['Tweet'].apply(lemmatization)
df_text.head()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\陳柏硯\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\陳柏硯\AppData\Roaming\nltk_data...
C:\Users\陳柏硯\AppData\Local\Temp\ipykernel_42156\1947433968.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['Tweet']=df_text['Tweet'].apply(lemmatization)


,Tweet
0,"[burn, None, None]"
1,"[vodka, knock, damn, tribe, solar, hemin, eerie]"
2,[bed]
3,"[felt, first, flash, violence, fool, bump, Non..."
4,"[lady, drink, get, free, till, None]"


In [9]:
# 詞幹提取
def stemming(text):
    porter = PorterStemmer()
    result=[]
    for word in text:
        result.append(porter.stem(word))
    return result

# 測試
text=['Connects','Connecting','Connections','Connected','Connection','Connectings','Connect']
stemming(text)

df_text['Tweet']=df_text['Tweet'].apply(stemming)
df_text.head()

C:\Users\陳柏硯\AppData\Local\Temp\ipykernel_42156\2327574843.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['Tweet']=df_text['Tweet'].apply(stemming)


,Tweet
0,"[burn, none, none]"
1,"[vodka, knock, damn, tribe, solar, hemin, eeri]"
2,[bed]
3,"[felt, first, flash, violenc, fool, bump, none..."
4,"[ladi, drink, get, free, till, none]"


In [10]:
# 移除HTML標籤
def remove_tag(text):
    text=' '.join(text)
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

# 測試
text = "<HEAD> this is head tag </HEAD>"
remove_tag(text.split())

# 移除URL
def remove_urls(text):
    if not isinstance(text, str):
        return text
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)
# 測試
text = "Machine learning knowledge is an awsome site. Here is the link for it https://machinelearningknowledge.ai/"
remove_urls(text)

df_text['Tweet'] = df_text['Tweet'].apply(remove_urls)
df_text.head()


C:\Users\陳柏硯\AppData\Local\Temp\ipykernel_42156\2682850534.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['Tweet'] = df_text['Tweet'].apply(remove_urls)


,Tweet
0,"[burn, none, none]"
1,"[vodka, knock, damn, tribe, solar, hemin, eeri]"
2,[bed]
3,"[felt, first, flash, violenc, fool, bump, none..."
4,"[ladi, drink, get, free, till, none]"


In [11]:
print(df_text['Tweet'])
df_text[["Tweet"]].to_csv('output_file.csv', index=False)

0                                       [burn, none, none]
1          [vodka, knock, damn, tribe, solar, hemin, eeri]
2                                                    [bed]
3        [felt, first, flash, violenc, fool, bump, none...
4                     [ladi, drink, get, free, till, none]
                               ...                        
10497          [none, sampan, none, none, none, hat, none]
10498    [el, lido, mica, north, de, paladin, main, non...
10499    [none, none, man, pah, love, terra, koto, coff...
10500             [none, batch, feder, lowest, hate, none]
10501    [manta, cater, none, naira, none, ana, honor, ...
Name: Tweet, Length: 10502, dtype: object
